In [2]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "20.0.1" 2023-04-18; OpenJDK Runtime Environment Homebrew (build 20.0.1); OpenJDK 64-Bit Server VM Homebrew (build 20.0.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/tmptiy71ty5
  JVM stdout: /var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/tmptiy71ty5/h2o_matiaslopez_started_from_python.out
  JVM stderr: /var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/tmptiy71ty5/h2o_matiaslopez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,H2O_from_python_matiaslopez_mopc0w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15.98 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
y = "Protestas"
balanced_df = pd.read_csv('../balanced_output_35.csv', index_col=False)
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(balanced_df, test_size=0.2, random_state=42)

## Random undersampling

In [4]:
# Random undersampling
if True:
	balanced_df_true = balanced_df[balanced_df[y] == True]
	balanced_df_false = balanced_df[balanced_df[y] == False]
	n_cmp = len(balanced_df_true)
	n_no_cmp = len(balanced_df_false)
	total = n_cmp + n_no_cmp
	sample_false = balanced_df_false.sample(int(total*.65)) #Sample 70% no complaints 30 % complaints
	balanced_df = pd.concat([balanced_df_true, sample_false])

## Training

In [5]:
train = h2o.H2OFrame.from_python(balanced_df)
# Identify predictors and response
x = train.columns
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
# Run AutoML for 10 base models
aml = H2OAutoML(max_models=42, seed=23, stopping_metric="AUCPR", sort_metric="AUCPR")
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leader
# board
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |
10:11:31.284: _train param, Dropping bad and constant columns: [parties.details.legalEntityTypeDetail payer_3, parties.details.legalEntityTypeDetail payer_23, parties.details.legalEntityTypeDetail payer_4, parties.details.legalEntityTypeDetail payer_24, tender.eligibilityCriteria q3, parties.details.legalEntityTypeDetail payer_1, parties.details.legalEntityTypeDetail payer_21, parties.details.legalEntityTypeDetail payer_2, parties.details.legalEntityTypeDetail payer_22, parties.details.legalEntityTypeDetail payer_7, parties.details.legalEntityTypeDetail payer_8, parties.details.legalEntityTypeDetail payer_5, parties.details.legalEntityTypeDetail payer_25, awards.documents.DocumentTypeDetails_19, parties.details.legalEntityTypeDetail payer_6, parties.details.legalEntityTypeDetail tenderer_18, tender.status_planning, parties.details.legalEntityTypeDetail payer_20, parties.details.legalEntityTypeDetail enquirer_18, awards.documents.DocumentTypeDetails_20, parties.detail

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20230808_101131,0.97273,0.982361,0.124364,0.0604357,0.182931,0.0334639
StackedEnsemble_BestOfFamily_1_AutoML_1_20230808_101131,0.972322,0.982095,0.125553,0.0621509,0.183865,0.0338065
XGBoost_grid_1_AutoML_1_20230808_101131_model_5,0.970176,0.980271,0.131939,0.0625483,0.186639,0.0348343
GBM_grid_1_AutoML_1_20230808_101131_model_7,0.969557,0.98001,0.134897,0.0639888,0.188556,0.0355535
GBM_grid_1_AutoML_1_20230808_101131_model_8,0.969458,0.980049,0.135767,0.0647377,0.189115,0.0357645
XGBoost_grid_1_AutoML_1_20230808_101131_model_7,0.967238,0.978236,0.139177,0.0655407,0.191504,0.036674
GBM_grid_1_AutoML_1_20230808_101131_model_5,0.966921,0.978952,0.144098,0.0699074,0.196237,0.0385088
GBM_grid_1_AutoML_1_20230808_101131_model_4,0.966593,0.978212,0.144799,0.0673295,0.195062,0.0380491
XGBoost_grid_1_AutoML_1_20230808_101131_model_8,0.966327,0.978567,0.142624,0.0700083,0.196918,0.0387767
XGBoost_grid_1_AutoML_1_20230808_101131_model_1,0.965602,0.977125,0.143941,0.0657154,0.195018,0.0380322


In [7]:
model_path = h2o.save_model(model=aml.leader, path="train2021/smoteModelv7", force=True)

In [8]:
print(model_path)

/Users/matiaslopez/Documents/tesis/tesis-model/Training/train2021/smoteModelv7/StackedEnsemble_AllModels_1_AutoML_1_20230808_101131


In [9]:
saved_model = h2o.load_model(model_path)

## Testing 2022 Data

In [10]:
df_2022 = pd.read_csv('../df_dummizado_from_2021.csv', index_col=False)
df_2022 = df_2022[df_2022['date.year'] == 2022]
test_2022 = h2o.H2OFrame.from_python(df_2022)
test_2022[y] = test_2022[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
def get_performance(model, test: h2o.H2OFrame):
	cm = model.model_performance(test).confusion_matrix().to_list()
	perf_t = cm[1][1] / (cm[1][0] + cm[1][1])
	perf_f = cm[0][0] / (cm[0][1] + cm[0][0])
	return perf_t, perf_f

In [12]:
best = ''
best_score_t = 0
best_score_f = 0
df_leaderboard = aml.leaderboard.as_data_frame()
model_ids = df_leaderboard['model_id']
for model_id in model_ids:
	model = h2o.get_model(model_id)
	score_t, score_f = get_performance(model, test_2022)
	print(model_id, score_t, score_f)
	if score_t > best_score_t and score_f > best_score_f:
		best_score_t = score_t
		best_score_f = score_f
		best = model_id

print(best, best_score_t, best_score_f)

StackedEnsemble_AllModels_1_AutoML_1_20230808_101131 0.9623287671232876 0.9935373073620464
StackedEnsemble_BestOfFamily_1_AutoML_1_20230808_101131 0.9594748858447488 0.9941248248745876
XGBoost_grid_1_AutoML_1_20230808_101131_model_5 0.9794520547945206 0.994667148732318
GBM_grid_1_AutoML_1_20230808_101131_model_7 0.944634703196347 0.9904189451800968
GBM_grid_1_AutoML_1_20230808_101131_model_8 0.9140981735159818 0.9922718850273422
XGBoost_grid_1_AutoML_1_20230808_101131_model_7 0.9480593607305936 0.9907804944185836
GBM_grid_1_AutoML_1_20230808_101131_model_5 0.827054794520548 0.9868034527952275
GBM_grid_1_AutoML_1_20230808_101131_model_4 0.9292237442922374 0.9887015862972839
XGBoost_grid_1_AutoML_1_20230808_101131_model_8 0.8410388127853882 0.984634157364306
XGBoost_grid_1_AutoML_1_20230808_101131_model_1 0.9275114155251142 0.9910516563474489
XGBoost_grid_1_AutoML_1_20230808_101131_model_3 0.8398972602739726 0.9760473629502417
XGBoost_grid_1_AutoML_1_20230808_101131_model_6 0.78995433789

In [15]:
best_model = h2o.get_model('XGBoost_grid_1_AutoML_1_20230808_101131_model_5')
best_model.model_performance(test_2022).confusion_matrix()

,False,True,Error,Rate
False,22009.0,118.0,0.0053,(118.0/22127.0)
True,72.0,3432.0,0.0205,(72.0/3504.0)
Total,22081.0,3550.0,0.0074,(190.0/25631.0)


In [16]:
best_model.model_performance(test_2022).confusion_matrix().to_list()

[[22009, 118], [72, 3432]]

In [17]:
model_path = h2o.save_model(model=best_model, path="train2021/smoteModelv8", force=True)

In [21]:
saved_model = h2o.load_model("/Users/matiaslopez/Documents/tesis/tesis-model/Training/train2021/smoteModelv8/XGBoost_grid_1_AutoML_1_20230808_101131_model_5")

In [22]:
saved_model

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_1_20230808_101131_model_5


Model Summary: 
    number_of_trees
--  -----------------
    44

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.005503288374358291
RMSE: 0.07418415177353106
LogLoss: 0.0328548694452838
Mean Per-Class Error: 0.00286308299666496
AUC: 0.9999544973111892
AUCPR: 0.9998929174170564
Gini: 0.9999089946223785

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35177770782919493
       False    True    Error    Rate
-----  -------  ------  -------  ----------------
False  73732    59      0.0008   (59.0/73791.0)
True   145      29287   0.0049   (145.0/29432.0)
Total  73877    29346   0.002    (204.0/103223.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.351778     0.996529  212
max f2                       0.278008     0.996598  225
max f0point5                 0.371526     0.997736  208
max accuracy                 0.351778     0.998024  212
max precision                0.999791     1         0
max recall                   0.0614746    1         310
max specificity              0.999791     1         0
max absolute_mcc             0.351778     0.99515   212
max min_per_class_accuracy   0.268574     0.997316  227
max mean_per_class_accuracy  0.293899     0.997451  221
max tns                      0.999791     73791     0
max fns                      0.999791     24577     0
max fps                      0.000266434  73791     399
max tps                      0.0614746    29432     310
max tnr                      0.999791     1         0
max fnr                      0.999791     0.835043  0
max fpr                      0.000266434  1         399
max tpr                      0.0614746    1         310

Gains/Lift Table: Avg response rate: 28.51 %, avg score: 28.53 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100171                   0.99989            3.50717     3.50717            1                0.999925     1                           0.999925            0.0351318       0.0351318                  250.717   250.717            0.0351318
2        0.0200149                   0.999824           3.50717     3.50717            1                0.999856     1                           0.99989             0.0350639       0.0701957                  250.717   250.717            0.0701957
3        0.030003                    0.999754           3.50717     3.50717            1                0.99979      1                           0.999857            0.0350299       0.105226                   250.717   250.717            0.105226
4        0.0400202                   0.999668           3.50717     3.50717            1                0.999714     1                           0.999821            0.0351318       0.140357                   250.717   250.717            0.140357
5        0.0500082                   0.99955            3.50717     3.50717            1                0.999614     1                           0.99978             0.0350299       0.175387                   250.717   250.717            0.175387
6        0.100007                    0.998239           3.50717     3.50717            1                0.999032     1                           0.999406            0.175353        0.350741                   250.717   250.717            0.350741
7        0.1500